# Lecture 2:
## Machine Learning Ising Model

E-Learning African International School on Quantum Science and Technology

Sept 25, 2024

Code by Lauren Hayward, Juan Carrasquilla, and Mohamed Hibat-Allah

## Problem set-up

The objective of this problem is to train a feedforward neural network to classifying ferromagnetic (FM) and paramagnetic (PM) phases
of the two-dimensional classical Ising model.
You are encouraged to start by reading Reference https://arxiv.org/pdf/1605.01735.

You have been given a dataset containing the following files:

- `x_L30.txt`: $10\,000$ samples of Ising spin configurations on a $30 \times 30$ lattice, which were generated using Monte Carlo sampling.
Each spin `up` state is stored as 1 and each spin `down` state is stored as 0. Note that this is different from what we have seen in the lectures. You can always relate to spins $\pm 1$ by a linear transformation of the data, but this is not necessary.

- `y_L30.txt`: The labels corresponding to each configuration.
Each label represents whether a configuration was generated with $T<T_{\text c}$ (label 0, corresponding to the FM phase in the thermodynamic limit)
or $T>T_{\text c}$ (label 1, corresponding to the PM phase in the thermodynamic limit).

- `T_L30.txt`: The temperatures corresponding to each sample (measured in units of the coupling $J$).
The temperature data file will not be used during the training, but will be used to study the accuracy of the network as a function of temperature in part 4).


You can download the data on Google Colab using the following line:

`!wget https://raw.githubusercontent.com/mhibatallah/ML-for-many-body-physics-course/main/FFNN_tutorial_data/filename.txt`

where `filename.txt` = `x_L30.txt`, `y_L30.txt`, or `T_L30.txt` taken from this paper: https://www.nature.com/articles/nphys4035 (https://arxiv.org/abs/1605.01735). The goal of this problem is to train a feedforward neural network to learn the labels of this dataset.

**Questions:**
1. Run the code below and try to make sence of the logic of the code. You can use the tool `Gemini` if you have a question about a certain line of code.

2. Plot accuracy and cost for training and validation data throughout the optimization using a FFNN with a preferred choice of hyperparameters. What is the accuracy on the testing data?

3. Study the effect of different hyperparameters:
  - Learning rate.
  - Optimizer (gradient descent with and without momentum, Adam, ...).
  - Activation functions.
  - Number of neurons in the hidden layer.
  - Number of hidden layers.

4. Plot accuracy and output neurons values as a function of temperature. Provide an interpretation of the results. Compare your plots with Figure 1 of Reference (https://arxiv.org/abs/1605.01735). Discuss the behaviour that you observe as you approach the critical temperature $T_{\text c}/J \approx 2.269$.


## Create and plot the data set


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
!wget https://raw.githubusercontent.com/mhibatallah/ML-for-many-body-physics-course/main/FFNN_tutorial_data/x_L30.txt
!wget https://raw.githubusercontent.com/mhibatallah/ML-for-many-body-physics-course/main/FFNN_tutorial_data/y_L30.txt
!wget https://raw.githubusercontent.com/mhibatallah/ML-for-many-body-physics-course/main/FFNN_tutorial_data/T_L30.txt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #To use GPUs in PyTorch
L=30
n0 = L*L
### Load data and shuffle it ###
x = np.loadtxt( 'x_L%d.txt' %L, dtype=np.int32 )
y = np.loadtxt( 'y_L%d.txt' %L, dtype=np.int32 )
T = np.loadtxt( 'T_L%d.txt' %L, dtype=np.float64 )
N_configs = x.shape[0]
indices_shuffled = np.random.permutation(N_configs)
x = x[indices_shuffled,:]
y = y[indices_shuffled]
T = T[indices_shuffled]

### Divide into training, validation and testing datasets ###
frac_train = 0.7 #Fraction of data used for training
frac_validation = 0.2 #Fraction of data used for validation
N_train = int(frac_train*N_configs)
N_validation = int(frac_validation*N_configs)
x_train = x[0:N_train,:]
y_train = y[0:N_train]
x_validation = x[N_train:(N_train+N_validation),:]
y_validation = y[N_train:(N_train+N_validation)]
x_test = x[(N_train+N_validation):,:]
y_test = y[(N_train+N_validation):]
T_test = T[(N_train+N_validation):]

## Define the network architecture and training hyperparameters

In [ ]:
%matplotlib inline
from IPython import display

import time

class FeedforwardNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardNN, self).__init__()

        #layer sizes:
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.output_size = output_size

        #functions used within the Feedforward NN:
        self.linear1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.linear2 = torch.nn.Linear(self.hidden_size, self.output_size)
        self.relu    = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax()
    def forward(self, x):
        #Layer 1:
        linear1_out = self.linear1(x)
        #a1 = self.sigmoid(linear1_out)
        a1 = self.relu(linear1_out)

        #Layer 2:
        linear2_out = self.linear2(a1)
        #a2 = self.sigmoid(linear2_out)
        a2 = self.softmax(linear2_out)

        #Network output:
        aL = a2

        return aL

input_size  = n0
output_size = 2
hidden_size = 4 #number of hidden units
model = FeedforwardNN(input_size, hidden_size, output_size).to(device)
### END OF MODIFICATIONS FOR PROBLEM 1c ###

### Store the input data as a PyTorch tensor ###
x_train = torch.tensor(x_train, dtype = torch.float).to(device) #add GPU support

### One hot encoding ###
y_onehot = np.zeros((y_train.size, output_size))
y_onehot[np.arange(y_train.size),y_train] = 1
y_onehot = torch.tensor(y_onehot, dtype = torch.float).to(device) #add GPU support

### Use backpropagation to minimize the cost function using the gradient descent algorithm: ###
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Cost function: ###
cost_func = torch.nn.MSELoss()
# cost_func = torch.nn.CrossEntropyLoss()

N_epochs = 10000 # number of times to run gradient descent

## Training

In [ ]:
epoch_list    = []
cost_training = []
acc_training  = []

############ Function for plotting: ############
def updatePlot():

    ### Generate coordinates covering the whole plane: ###
    padding = 0.1
    spacing = 0.02
    # x1_min, x1_max = x_train[:, 0].min() - padding, x_train[:, 0].max() + padding
    # x2_min, x2_max = x_train[:, 1].min() - padding, x_train[:, 1].max() + padding
    # x1_grid, x2_grid = np.meshgrid(np.arange(x1_min, x1_max, spacing),
    #                      np.arange(x2_min, x2_max, spacing))

    #torch_input = torch.tensor(np.c_[x1_grid.ravel(), x2_grid.ravel()], dtype = torch.float)
    NN_output = model(x_train)
    predicted_class = np.argmax(NN_output.cpu().detach().numpy(), axis=1)

    ### Plot the classifier: ###
    #plt.subplot(121)
    #plt.contourf(x1_grid, x2_grid, predicted_class.reshape(x1_grid.shape), K, alpha=0.8)
    #plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train, s=40)
    #plt.xlim(x1_grid.min(), x1_grid.max())
    #plt.ylim(x2_grid.min(), x2_grid.max())
    #plt.xlabel(r'$x_1$')
    #plt.ylabel(r'$x_2$')

    ### Plot the cost function during training: ###
    plt.subplot(222)
    plt.plot(epoch_list,cost_training,'o-')
    plt.xlabel('Epoch')
    plt.ylabel('Training cost')

    ### Plot the training accuracy: ###
    plt.subplot(224)
    plt.plot(epoch_list,acc_training,'o-')
    plt.xlabel('Epoch')
    plt.ylabel('Training accuracy')
############ End of plotting function ############

### Train for several epochs: ###
for epoch in range(N_epochs):

    optimizer.zero_grad() # sets the gradients to zero (necessary since PyTorch accumulates the gradients)
    NN_output = model(x_train) # Neural network output
    cost = cost_func(NN_output, y_onehot)
    cost.backward() #computes the gradients
    optimizer.step() #updating the parameters

    ### Update the plot and print results every 500 epochs: ###
    if epoch % 500 == 0:
        predicted_class = np.argmax(NN_output.cpu().detach().numpy(), axis=1)
        accuracy = np.mean(predicted_class == y_train)

        epoch_list.append(epoch)
        cost_training.append(cost.cpu().detach().numpy())
        acc_training.append(accuracy)

        ### Update the plot of the resulting classifier: ###
        fig = plt.figure(2,figsize=(10,5))
        fig.subplots_adjust(hspace=.3,wspace=.3)
        plt.clf()
        updatePlot()
        display.display(plt.gcf())
        print("Iteration %d:\n  Training cost %f\n  Training accuracy %f\n" % (epoch, cost, accuracy) )
        display.clear_output(wait=True)
        # time.sleep(0.1) #Uncomment this line if you want to slow down the rate of plot updates

plt.savefig('results.pdf', bbox_inches="tight")
print("Final Training cost %f\nFinal Training accuracy %f\n" % (cost, accuracy) )

In [ ]:
### TESTING DATA ###
### Store the input data as a PyTorch tensor ###
x_test = torch.tensor(x_test, dtype = torch.float).to(device) #add GPU support

### One hot encoding ###
y_test_onehot = np.zeros((y_test.size, output_size))
y_test_onehot[np.arange(y_test.size),y_test] = 1
y_test_onehot = torch.tensor(y_test_onehot, dtype = torch.float).to(device) #add GPU support

NN_output_test = model(x_test)
predicted_class_test = np.argmax(NN_output_test.cpu().detach().numpy(), axis=1)
accuracy_test = np.mean(predicted_class_test == y_test)
print(accuracy_test)